# Programming for Probability

#### Authors:
v1.0 (Spring 2020) William Gan, Aditya Sengupta, Christina Zhang, Sasha Khazatsky, Kannan Ramchandran

v1.1 (Spring 2023) Andy Dong, Thomas Courtade

## Introduction

What is the probability that a 5 card hand in poker is a flush? What is the expected number of times you have to flip a coin until you see three heads in a row? What is the optimal way to play blackjack at a casino?

In EECS 126, we teach you techniques to answer these questions with just pencil and paper. However, in the real world, it can be useful to simply use computers to calculate or approximate these answers. In this lab, we give an overview of various techniques to do so and give you some practice.

In [1]:
import itertools

import numpy as np

## Q1: Brute Force / Exhaustive Search

Sometimes it's possible to iterate through every outcome. For example, to find the probability that a 5 card hand in poker is a flush, maybe we can just iterate over every possible 5 card hand and then see how many are flushes. Since every 5 card hand is equally likely, the probability can be found via simple division.

**Question: How many 5 card hands are there (exact number)?**

$$\binom{52}{5} = 2598960$$

**Fill in the following functions that are used to calculate if a hand is a flush.** A 5-card hand is a flush if all the suits are the same and it is not a straight. A straight consists of 5 consecutive ranks e.g. 8 9 10 J Q. Ranks are not allowed to wrap around (e.g. K A 2 3 4) is not a straight; however, the Ace **is** allowed to be at the front or back. For example, 10 J Q K A and A 2 3 4 5 are both straights. You can run `test_is_consecutive` to make sure it works.

In [2]:
RANKS = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']
SUITS = ['C', 'D', 'H', 'S']

In [3]:
def is_same_suit(hand):
    ### BEGIN YOUR SOLUTION
    suit = hand[0][1]
    for card in hand:
        if not card[1] == suit:
            return False
    return True
    ### END YOUR SOLUTION
    
def is_consecutive(hand):
    ### BEGIN YOUR SOLUTION
    assert len(hand) == 5
    numerical_ranks = [RANKS.index(card[0])+2 for card in hand]
    sorted_ranks = sorted(numerical_ranks)
    if sorted_ranks == [2, 3, 4, 5, 14]:
        return True
    for i in range(4):
        if not sorted_ranks[i+1] == sorted_ranks[i]+1:
            return False
    return True
    ### END YOUR SOLUTION

def is_flush(hand):
    assert len(hand) == 5
    return is_same_suit(hand) and not is_consecutive(hand)

In [4]:
def test_is_consecutive():
    assert is_consecutive(((8, 'C'), (9, 'D'), (10, 'H'), ('J', 'S'), ('Q', 'C')))
    assert not is_consecutive(((8, 'C'), (9, 'D'), (2, 'H'), ('J', 'S'), ('Q', 'C')))
    assert is_consecutive(((10, 'C'), ('J', 'C'), ('Q', 'C'), ('K', 'C'), ('A', 'C')))
    assert is_consecutive((('A', 'H'), (2, 'H'), (3, 'H'), (4, 'D'), (5, 'C')))
    assert not is_consecutive((('K', 'H'), ('A', 'H'), (2, 'H'), (3, 'D'), (4, 'C')))
    print('All tests passed')

In [5]:
test_is_consecutive()

All tests passed


`itertools` is a built-in Python library that provides Python generators for things like permutations and combinations. For example, `itertools.product(RANKS, SUITS)` returns the tuples (2, 'C'), (2, 'D'), ..., ('A', 'S'). **Use it to iterate through every 5 card hand and calculate the probability of a flush.**

Hint: Look through the Python documentation to find `itertools.combinations`.

In [6]:
deck = itertools.product(RANKS, SUITS)
total = 0
flushes = 0

### BEGIN YOUR SOLUTION
for hand in itertools.combinations(deck, r=5):
    if is_flush(hand):
        flushes += 1
    total += 1
### END YOUR SOLUTION

print(flushes / total)

0.001965401545233478


## Q2: Monte Carlo Simulation

Sometimes there are too many outcomes to iterate over. While a single 5-card hand is relatively simple, a multi-player poker game can have billions of outcomes. Even if you were to group some outcomes based on symmetry, there would still be too many. Some problems, like finding how long it takes to see three heads in a row, have an infinite number of outcomes.

In these situations, we can still find approximate answers via sampling. For example, to find how long it takes to see three heads in a row, maybe we can just play it out many times and take the average.

To run this simulation, we need to be able to generate random events. Python's built-in `random` module allows you to do this, but usually we want to use `np.random`. They have more or less the same functionality, but using NumPy will sometimes have a performance advantage.

**In the following cell, use Monte Carlo sampling to calculate how many flips are needed to see three heads in a row.**

In [7]:
TRIALS = 100000
running_sum = 0
for _ in range(TRIALS):
    ### BEGIN YOUR SOLUTION
    num_heads_in_a_row = 0
    num_flips = 0
    while True:
        if np.random.randint(2) == 0:
            num_heads_in_a_row = 0
        else:
            num_heads_in_a_row += 1
        num_flips += 1
        if num_heads_in_a_row == 3:
            break
    running_sum += num_flips
    ### END YOUR SOLUTION
print(f'The empirical average of the number of flips required to see three heads in a row is {running_sum / TRIALS}')

The empirical average of the number of flips required to see three heads in a row is 13.97966


### Q3: Birthday Paradox

How many people do you think there need to be in a room for two of them to have the same birthday with probability at least 50%? Someone who hasn't taken EECS 126 might say $\frac{365}{2} \approx 180$, but it turns out the answer is a lot fewer. We can approximate what this answer is using the following code.

**In the following cell, use Monte Carlo sampling to determine the probability that two people have the same birthday.** You can use `test_same_birthday` to verify the probabilities.

In [8]:
def p_same_birthday(num_people, trials=10000):
    same_birthday_count = 0
    ### BEGIN YOUR SOLUTION
    for _ in enumerate(range(trials)):
        birthdays = np.random.randint(365, size=num_people)
        if len(np.unique(birthdays)) < num_people:
            same_birthday_count += 1
    ### END YOUR SOLUTION
    return same_birthday_count / trials

In [9]:
def test_p_same_birthday():
    assert np.abs(p_same_birthday(10) - 0.11694817771107768) < 0.02
    assert np.abs(p_same_birthday(40) - 0.891231809817949) < 0.02
    print('All tests passed')

In [10]:
test_p_same_birthday()

All tests passed


**Run the following cell to find the smallest value of `num_people` such that the Monte Carlo frequency of a birthday collision is at least 0.5.**

In [11]:
num_people = 2
while True:
    if p_same_birthday(num_people) > 0.5:
        print(f'In a room of {num_people} people, the probability of at least a pair having the same birthday is > 0.5')
        break
    else:
        num_people += 1

In a room of 23 people, the probability of at least a pair having the same birthday is > 0.5


## Q4: Tricks

The message so far has basically been that for simple problems, we can just use brute force, and for harder problems, we can perform sampling. This is mostly true, but there is often an art that comes to both techniques. In many cases, you can exploit symmetries or be clever in other ways. We already did this with the 5-card flush probability: a more naive way would have been to iterate over all $52 \cdot 51 \cdot 50 \cdot 49 \cdot 48$ ways 5-cards could be dealt. Most "tricks" aren't as obvious, but when found, they can greatly speed up your program.

Say we wanted to find the probability a 12-card hand is "symmetrical". A hand is called symmetrical if it can be arranged in a way that is symmetrical. For 5-card hands, A K K 5 A is symmetrical since it can be arranged into A K 5 K A, whereas 10 10 J Q 10 isn't symmetrical. To find this probability, we could take a similar approach to the flush program, but that would involve going through 206379406870 hands. This isn't feasible, but it turns out some hands are the same, since in this problem, the suit doesn't matter. A hand is really just a tuple with the counts of each rank. For example 10 10 J Q 10 is

2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | J | Q | K | A
- | - | - | - | - | - | - | - | -- | - | - | - | -
0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 3  | 1 | 1 | 0 | 0

And the number of unique tuples is far lower. So what we need to do is iterate through all the unique tuples. However, we also need to explicitly find the probability of each tuple, since they are not all the same.

You may recall this practice as the Law of the Unconscious Statistician (LOTUS) from the homework, where we move to a probability space that is easier to work with and "push-forward" the law of the random variable of interest.

**Question: Assume we can still distinguish cards by suit. How many 5-card combinations have 3 10s, 1 J, and 1 Q?**

$$\binom{4}{3} \binom{4}{1} \binom{4}{1} = 64$$

We are going to use the unique tuple approach to calculate the probability that a 12-card hand is symmetrical. **Complete the following function that determines if a tuple represents a hand that is symmetrical**. Your function must be general and work with any hand size (not just 12). You can use `test_is_symmetrical` to sanity-check your implementation.

In [12]:
def is_symmetrical(hand_tuple):
    assert len(hand_tuple) == 13
    ### BEGIN YOUR SOLUTION
    num_odd = 0
    for count in hand_tuple:
        if count % 2 == 1:
            num_odd += 1
    return num_odd <= 1
    ### END YOUR SOLUTION

In [13]:
def test_is_symmetrical():
    assert is_symmetrical((1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))
    assert is_symmetrical((2, 0, 4, 0, 2, 0, 0, 1, 0, 4, 4, 0, 2))
    assert is_symmetrical((0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0))
    assert not is_symmetrical((2, 0, 3, 0, 2, 0, 0, 1, 0, 4, 4, 0, 2))
    assert not is_symmetrical((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0))
    print('All tests passed')

In [14]:
test_is_symmetrical()

All tests passed


Now we need to calculate the probability of a hand_tuple. However, for numerical precision, we may just want to keep track of the total number of card combinations to get a tuple. In the end, we can divide this by the number of possible 12-card hands to find the probability. **Complete the following function that calculates this**. You can use `test_combinations` to verify your implementation is correct.

In [15]:
def combinations(hand_tuple):
    assert len(hand_tuple) == 13
    ### BEGIN YOUR SOLUTION
    numerator = 1
    TABLE = [1, 4, 6, 4, 1]
    for count in hand_tuple:
        numerator *= TABLE[count]
    return numerator
    ### END YOUR SOLUTION

In [16]:
def test_combinations():
    assert combinations((1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)) == 4
    assert combinations((2, 0, 3, 0, 2, 0, 0, 1, 0, 4, 4, 0, 2)) == 3456
    assert combinations((1, 3, 3, 4, 2, 2, 1, 1, 0, 0, 0, 2, 1)) == 884736
    print('All tests passed')

In [17]:
test_combinations()

All tests passed


What remains is to write a Python function that can provide all unique tuples. This is a bit tricky, but can be done with the following idea. We recursively go through each index 0 to 12 in the tuple. At each index, we can decide to add 1 to that index or decide to move on to the next index. We just need to make sure that we only take at most 4 cards from each index and have 12 cards in total. You may recall a similar problem if you have taken CS 61A.

**Complete the following function**. You can use `test_unique_tuples` to verify your implementation is correct. If you wish, you may also write your own function from scratch.

In [18]:
def unique_tuples(hand_size):
    # returns all unique tuples of the desired hand size
    return unique_tuples_helper(hand_size, 0, [0 for x in range(13)])

def unique_tuples_helper(hand_size, index, hand_array):
    # a helper function that returns all unique tuples of the desired hand size.
    # hand_array is the current (uncompleted) configuration of tuple.
    # Any tuple that is returned by this helper must satisfy these requirements:
    # 1. For all indices smaller than index, the entry at that index in the tuple must be the same
    # value as the entry at that index in hand_array,
    # 2. For all indices greater than or equal to index, the entry at that index in the tuple must
    # be at least the value at that index in hand_array
    num_cards = sum(hand_array)
    if num_cards == hand_size:
        return [tuple(hand_array)]
    if index >= 13:
        return []
    
    all_tuples = []
    
    ### BEGIN YOUR SOLUTION
    if hand_array[index] < 4:
        hand_array_copy = hand_array.copy()
        hand_array_copy[index] += 1
        all_tuples.extend(unique_tuples_helper(hand_size, index, hand_array_copy))
    
    max_cards_left = (12-index) * 4
    if num_cards + max_cards_left >= hand_size:
        all_tuples.extend(unique_tuples_helper(hand_size, index+1, hand_array))
                          
    ### END YOUR SOLUTION
    
    return all_tuples

In [19]:
def test_unique_tuples():
    total_combinations = 0
    total_tuples = 0
    for hand_tuple in unique_tuples(6):
        total_combinations += combinations(hand_tuple)
        total_tuples += 1
    assert total_tuples == 18395
    assert total_combinations == 20358520 
    print('All tests passed')

In [20]:
test_unique_tuples()

All tests passed


In [21]:
TOTAL = 206379406870
total_combinations = 0
for hand_tuple in unique_tuples(12):
    if is_symmetrical(hand_tuple):
        total_combinations += combinations(hand_tuple)
print(total_combinations / TOTAL)

0.00042909408134786544
